In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from goose3 import Goose
import os
from nltk.tokenize import word_tokenize
import nltk
import re
from nltk.corpus import stopwords

In [2]:
goose = Goose()

In [3]:
data = pd.read_excel('Input.xlsx')
data.shape

(114, 2)

In [4]:
data.head()

,URL_ID,URL
0,123.0,https://insights.blackcoffer.com/rise-of-telem...
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...
4,432.0,https://insights.blackcoffer.com/rise-of-telem...


In [5]:
article_title = []
article_text = []
for i in range(114):
    article = goose.extract(data['URL'][i])
    article_title.append(article.title)
    article_text.append(article.cleaned_text.replace('\n',"").replace('"',''))

In [6]:
data['Title'] = article_title
data['Text'] = article_text

In [7]:
def token(text):
    tokens = re.split('\W+',text)
    return tokens

In [8]:
data['cleaned_text'] = data['Text'].apply(lambda x: token(x.lower()))

In [9]:
input_files = ['StopWords_Auditor.txt', 'StopWords_Currencies.txt', 'StopWords_DatesandNumbers.txt','StopWords_Generic.txt','StopWords_GenericLong.txt','StopWords_Geographic.txt','StopWords_Names.txt']
stopword = []
for file_name in input_files:
    with open(file_name, 'r') as input_file:
        for word in input_file:
            stopword.append(word.replace(' \n','').replace('|','').replace("\n",""))

In [10]:
data.head()

,URL_ID,URL,Title,Text,cleaned_text
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,Rise of telemedicine and its Impact on Livelih...,"Telemedicine, the use of technology to diagnos...","[telemedicine, the, use, of, technology, to, d..."
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,Rise of e-health and its impact on humans by t...,"The rise of e-health, or the use of electronic...","[the, rise, of, e, health, or, the, use, of, e..."
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,Rise of e-health and its impact on humans by t...,2020 was the year the world was ravaged by the...,"[2020, was, the, year, the, world, was, ravage..."
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,Rise of telemedicine and its Impact on Livelih...,"“More gains on quality, affordability and acce...","[, more, gains, on, quality, affordability, an..."
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,Rise of telemedicine and its Impact on Livelih...,"“More gains on quality, affordability and acce...","[, more, gains, on, quality, affordability, an..."


In [11]:
def remove_stopwords(text):
    stop_words = set(stopword)
    words = nltk.word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)



data['cleaned_text'] = data['Text'].apply(lambda x: remove_stopwords(x))


In [12]:
positive_words = []
negative_words = []
with open('positive-words.txt', 'r') as file:
    for word in file:
        positive_words.append(word.replace('\n',""))
with open('negative-words.txt','r') as w:
    for word in w:
        negative_words.append(word.replace('\n',""))

In [13]:

from nltk.tokenize import sent_tokenize, word_tokenize

def avg_sentence_length(text):
    sentences = sent_tokenize(text)
    sentence_word_counts = [len(word_tokenize(sentence)) for sentence in sentences]
    total_words = sum(sentence_word_counts)
    total_sentences = len(sentences)
    
    if total_sentences > 0:
        avg_length = total_words / total_sentences
    else:
        avg_length = 0

    return avg_length

data['Average Sentence length']  = data['Text'].apply(lambda x: avg_sentence_length(x))

In [14]:
def total_sentence(text):
    sentences = sent_tokenize(text)
    sentence_word_counts = [len(word_tokenize(sentence)) for sentence in sentences]
    total_words = sum(sentence_word_counts)
    total_sentences = len(sentences)

    return total_sentences


data['Sentence Count'] = data['Text'].apply(lambda x: total_sentence(x))

In [15]:
def count_words(text):
    words = text.split() 
    return len(words)

data['Number of words'] = data['cleaned_text'].apply(lambda x: count_words(x))

In [16]:
def count_positive_words(text):
    words = text.split() 
    positive_word_count = sum(1 for word in words if word.lower() in positive_words)
    return positive_word_count


def count_negative_words(text):
    words = text.split()  
    positive_word_count = sum(1 for word in words if word.lower() in negative_words)
    return positive_word_count

In [17]:
data['POSITIVE SCORE'] = data['cleaned_text'].apply(lambda x: count_positive_words(x))

In [18]:
data['NEGATIVE SCORE'] = data['cleaned_text'].apply(lambda x: count_negative_words(x))

In [19]:
data['POLARITY SCORE'] = (data['POSITIVE SCORE'] - data['NEGATIVE SCORE'])/((data['POSITIVE SCORE']+data['NEGATIVE SCORE'])+0.000001)

In [20]:
l = []
for i in range(len(data)):
    l.append(len(data['cleaned_text'][i]))
    
data['Words Length'] = l

avg_sentence_count = data['Sentence Count'].mean()

In [21]:
data['SUBJECTIVITY SCORE'] = (data['POSITIVE SCORE']+data['NEGATIVE SCORE'])/((data['Words Length'])+0.000001)

In [22]:
data['AVG SENTENCE LENGTH'] = data['Number of words']/data['Sentence Count']

In [28]:
import syllables
def count_complex_words(text):
    words = text.split()  # Split the text into words
    complex_word_count = sum(1 for word in words if syllables.estimate(word) > 2)
    return complex_word_count


data['Complex word'] = data['cleaned_text'].apply(count_complex_words)

In [32]:
data['PERCENTAGE OF COMPLEX WORDS'] = data['Complex word']/data['Number of words']

In [33]:
data['COMPLEX WORD COUNT'] = data['Complex word']/data['Number of words']

In [34]:
data['FOG INDEX'] = 0.4 * (data['AVG SENTENCE LENGTH']+data['PERCENTAGE OF COMPLEX WORDS'])

In [35]:
data['AVG NUMBER OF WORDS PER SENTENCE'] = data['Number of words']/data['Sentence Count']

In [36]:
import string
stop_words = set(nltk.corpus.stopwords.words('english'))
def count_cleaned_words(text):
    words = nltk.word_tokenize(text)
    cleaned_words = [word.lower() for word in words if word.lower() not in stop_words and word not in string.punctuation]
    return len(cleaned_words)

data['WORD COUNT'] = data['cleaned_text'].apply(count_cleaned_words)

In [37]:
def count_syllables(word):
    word = word.lower()
    if word.endswith("es") or word.endswith("ed"):
        word = word[:-2]
    vowels = "aeiouy"
    syllable_count = 0
    prev_char = ""
    for char in word:
        if char in vowels and prev_char not in vowels:
            syllable_count += 1
        prev_char = char
    if syllable_count == 0:
        syllable_count = 1

    return syllable_count

def count_total_syllables(text):
    words = nltk.word_tokenize(text)
    syllable_counts = [count_syllables(word) for word in words]
    return sum(syllable_counts)


data['SYLLABLE PER WORD'] = data['cleaned_text'].apply(count_total_syllables)

In [38]:
pronoun_pattern = r'\b(?:I|we|my|ours|us)\b'

def count_personal_pronouns(text):
    pronouns = re.findall(pronoun_pattern, text, re.IGNORECASE)
    pronouns = [pronoun for pronoun in pronouns if pronoun.lower() != 'us']
    return len(pronouns)


data['PERSONAL PRONOUNS'] = data['Text'].apply(count_personal_pronouns)

In [39]:
data['AVG WORD LENGTH'] = data['Words Length']/data['WORD COUNT']

In [40]:
data.head()

,URL_ID,URL,Title,Text,cleaned_text,Average Sentence length,Sentence Count,Number of words,POSITIVE SCORE,NEGATIVE SCORE,...,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,Complex word,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,Rise of telemedicine and its Impact on Livelih...,"Telemedicine, the use of technology to diagnos...","Telemedicine , technology diagnose treat patie...",31.892857,56,990,75,22,...,17.678571,0.472727,7.260519,17.678571,468,0.472727,843,2392,2,9.323843
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,Rise of e-health and its impact on humans by t...,"The rise of e-health, or the use of electronic...","rise e-health , electronic means facilitate he...",33.578947,19,321,38,13,...,16.894737,0.545171,6.975963,16.894737,175,0.545171,278,742,3,9.525180
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,Rise of e-health and its impact on humans by t...,2020 was the year the world was ravaged by the...,2020 year world ravaged SarsCov2 virus . notor...,18.873016,63,660,21,27,...,10.476190,0.381818,4.343203,10.476190,252,0.381818,543,1290,3,8.769797
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,Rise of telemedicine and its Impact on Livelih...,"“More gains on quality, affordability and acce...","“ gains quality , affordability accessibility ...",28.217391,46,740,33,27,...,16.086957,0.420270,6.602891,16.086957,311,0.420270,634,1543,7,8.820189
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,Rise of telemedicine and its Impact on Livelih...,"“More gains on quality, affordability and acce...","“ gains quality , affordability accessibility ...",28.217391,46,740,33,27,...,16.086957,0.420270,6.602891,16.086957,311,0.420270,634,1543,7,8.820189


In [41]:
data.columns.tolist()

['URL_ID',
 'URL',
 'Title',
 'Text',
 'cleaned_text',
 'Average Sentence length',
 'Sentence Count',
 'Number of words',
 'POSITIVE SCORE',
 'NEGATIVE SCORE',
 'POLARITY SCORE',
 'Words Length',
 'SUBJECTIVITY SCORE',
 'AVG SENTENCE LENGTH',
 'PERCENTAGE OF COMPLEX WORDS',
 'FOG INDEX',
 'AVG NUMBER OF WORDS PER SENTENCE',
 'Complex word',
 'COMPLEX WORD COUNT',
 'WORD COUNT',
 'SYLLABLE PER WORD',
 'PERSONAL PRONOUNS',
 'AVG WORD LENGTH']

In [42]:
data = data[['URL_ID',
 'URL',
 'Title',
 'Text',
 'POSITIVE SCORE',
 'NEGATIVE SCORE',
 'POLARITY SCORE',
 'SUBJECTIVITY SCORE',
 'AVG SENTENCE LENGTH',
 'PERCENTAGE OF COMPLEX WORDS',
    'FOG INDEX',
 'AVG NUMBER OF WORDS PER SENTENCE',
    'COMPLEX WORD COUNT',
 'WORD COUNT',
 'SYLLABLE PER WORD',
 'PERSONAL PRONOUNS',
 'AVG WORD LENGTH']]

In [43]:
data.head()

,URL_ID,URL,Title,Text,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,Rise of telemedicine and its Impact on Livelih...,"Telemedicine, the use of technology to diagnos...",75,22,0.546392,0.012341,17.678571,0.472727,7.260519,17.678571,0.472727,843,2392,2,9.323843
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,Rise of e-health and its impact on humans by t...,"The rise of e-health, or the use of electronic...",38,13,0.490196,0.019260,16.894737,0.545171,6.975963,16.894737,0.545171,278,742,3,9.525180
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,Rise of e-health and its impact on humans by t...,2020 was the year the world was ravaged by the...,21,27,-0.125000,0.010080,10.476190,0.381818,4.343203,10.476190,0.381818,543,1290,3,8.769797
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,Rise of telemedicine and its Impact on Livelih...,"“More gains on quality, affordability and acce...",33,27,0.100000,0.010730,16.086957,0.420270,6.602891,16.086957,0.420270,634,1543,7,8.820189
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,Rise of telemedicine and its Impact on Livelih...,"“More gains on quality, affordability and acce...",33,27,0.100000,0.010730,16.086957,0.420270,6.602891,16.086957,0.420270,634,1543,7,8.820189
